# 3

Vaccination över tid i riket uppdelat på dos

Strategi: 
- läsa in alla DataFrames
- separera minst dos 1 och minst dos 2 i olika DataFrames, resterande är redan separerade
- plocka ut endast rader för Sverige från varje DataFrame
- slå ihop tidskolumnerna 
- lägga ihop data för dos 1, dos2, dos 3, dos 4, dos 5 i samma DataFrame
- göra om tidsräkningen där v52 2020 sätts som 0
- plotta

In [1]:
import pandas as pd

# readin in relevant sheets into separate DataFrames
path = "../Lab/Data/vaccin.xlsx"
df_1_2 = pd.read_excel(path, sheet_name="Vaccinerade tidsserie")
df_3 = pd.read_excel(path, sheet_name="Vaccinerade tidsserie dos 3") # ???????? hur automatisera inläsning
df_4 = pd.read_excel(path, sheet_name="Vaccinerade tidsserie dos 4")
df_5 = pd.read_excel(path, sheet_name="Vaccinerade tidsserie dos 5")
df_1_2.head()

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
0,52,2020,| Sverige |,1639,0.000180,Minst 1 dos
1,52,2020,| Sverige |,0,0.000000,Minst 2 doser
2,52,2020,Stockholm,201,0.000096,Minst 1 dos
3,52,2020,Stockholm,0,0.000000,Minst 2 doser
4,52,2020,Uppsala,5,0.000015,Minst 1 dos


In [2]:
# devide df_1_2 into separate dataframes for Minst 1 dos och Minst 2 doser
df_1 = pd.DataFrame(df_1_2[df_1_2["Vaccinationsstatus"].str.contains("1")]) #??? måste jag göra om til DF manuellt????
df_2 = pd.DataFrame(df_1_2[df_1_2["Vaccinationsstatus"].str.contains("2")])
df_2.head()

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
1,52,2020,| Sverige |,0,0.0,Minst 2 doser
3,52,2020,Stockholm,0,0.0,Minst 2 doser
5,52,2020,Uppsala,0,0.0,Minst 2 doser
7,52,2020,Södermanland,0,0.0,Minst 2 doser
9,52,2020,Östergötland,0,0.0,Minst 2 doser


In [3]:
# funkar men den nya df sparas inte någonstans så kan inte anropa df utan måste anropa df_list[index i listan]
#
# df_list = [df_1, df_2, df_3, df_4, df_5]
# for df in df_list:
    #df = df[df["Region"].str.contains("Sverige")]
    
# df_list[4]

In [4]:
def clean(df):
    df = df[df["Region"].str.contains("Sverige")]                               # add comment
    df["År_v"] = df["År"].astype(str) + "v" + df["Vecka"].astype(str)           # combine colls Vecka and År to new column
    df = df.set_index("År_v")                                                   # set År_v col as index to prepare concatination
    df = df.rename(columns={"Antal vaccinerade": df["Vaccinationsstatus"][0]}) # rename column Antal vaccinerade with values for Vaccinationsstatus
    return df


In [5]:
#df_1_5 = pd.concat((clean(df) for df in [df_1, df_2, df_3, df_4, df_5]), axis = "columns")

In [7]:
# concatinating all DataFrames into 1 based on col "År_v"
df_list = [df_1, df_2, df_3, df_4, df_5]
df_list = [clean(df) for df in df_list]

df_1_5 = pd.concat(df_list, axis = "columns")
df_1_5.columns

C:\Users\DorotaBjöörn-AI22GBG\AppData\Local\Temp\ipykernel_2376\839915847.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["År_v"] = df["År"].astype(str) + "v" + df["Vecka"].astype(str)           # combine colls Vecka and År to new column
C:\Users\DorotaBjöörn-AI22GBG\AppData\Local\Temp\ipykernel_2376\839915847.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["År_v"] = df["År"].astype(str) + "v" + df["Vecka"].astype(str)           # combine colls Vecka and År to new column
C:\Users\DorotaBjöörn-

Index(['Vecka', 'År', 'Region', 'Minst 1 dos', 'Andel vaccinerade',
       'Vaccinationsstatus', 'Vecka', 'År', 'Region', 'Minst 2 doser',
       'Andel vaccinerade', 'Vaccinationsstatus', 'Vecka', 'År', 'Region',
       '3 doser', 'Andel vaccinerade', 'Vaccinationsstatus', 'Vecka', 'År',
       'Region', '4 doser', 'Andel vaccinerade', 'Vaccinationsstatus', 'Vecka',
       'År', 'Region', '5 doser', 'Andel vaccinerade', 'Vaccinationsstatus'],
      dtype='object')

In [18]:
# selecting relevant columns; replacing NaN with 0
# resetting År_v as column, creating column index corresponding to weeks from start
# renaming column index

col_list = ['Minst 1 dos', 'Minst 2 doser', '3 doser', '4 doser', '5 doser']

df_1_5 = df_1_5[col_list].fillna(0).reset_index().reset_index().rename(columns={"index": "Veckor från start"})

df_1_5["Totalt vaccinerade"] = sum(df_1_5[col] for col in col_list)
#df_1_5["Totalt vaccinerade"] = sum(df_1_5.iloc[:, 2:6]) detta funkar inte
df_1_5

,level_0,Veckor från start,Minst 1 dos,Minst 2 doser,3 doser,4 doser,5 doser,Totalt vaccinerade
0,0,0,1639,0,0.0,0.0,0.0,1639.0
1,1,1,11446,0,0.0,0.0,0.0,11446.0
2,2,2,37904,0,0.0,0.0,0.0,37904.0
3,3,3,104762,68,0.0,0.0,0.0,104830.0
4,4,4,201575,7995,0.0,0.0,0.0,209570.0
...,...,...,...,...,...,...,...,...
91,91,91,7806988,7623561,5509988.0,2078006.0,693794.0,23712337.0
92,92,92,7808072,7624734,5516559.0,2191428.0,837547.0,23978340.0
93,93,93,7808932,7625775,5522022.0,2285018.0,942467.0,24184214.0
94,94,94,7809738,7626723,5527028.0,2363358.0,1024426.0,24351273.0


In [19]:
import plotly_express as px

#nedanstående 3 rader behövs för att kunna importera functioner från dokumentet functions.py som ligger i Excercises
import sys
sys.path.append ("../Excercises/")
import functions # alt kan skriva from functions import remove_legend_heading

fig = px.line(
    df_1_5,
    x="Veckor från start",
    y=col_list, #???????????hitta ett sätt att lägga på kolumnen Totalt vaccinerade till y eller om ny y axel skall skapas??????
    title="Antal vaccinerade i Sverige per dos från vaccineringsstart v52 2020 till v42 2022",
    labels={"value" : "Ackumulerat antal vaccinerade"}
)
functions.remove_legend_heading(fig) #funktion remove_legend_heading skriven i dokumentet functions.py under Excercises
fig.show()

fig.write_html("../Lab/Visualiseringar/plot_3_3.html")